In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Retrieve Data

In [2]:
from stockMarket.stockData import StockData

stock_data = StockData("S&P 500")
stock_data.init_data()

100%|██████████| 503/503 [00:00<00:00, 104587.29it/s]


Retrieve info data from Yahoo Finance...


100%|██████████| 503/503 [01:08<00:00,  7.37it/s]


Retrieve cashflow data from Yahoo Finance...


100%|██████████| 503/503 [01:12<00:00,  6.89it/s]


...

100%|██████████| 503/503 [01:06<00:00,  7.58it/s]


Retrieve balancesheet data from Yahoo Finance...


100%|██████████| 503/503 [01:14<00:00,  6.76it/s]


Retrieve calendar data from Yahoo Finance...


100%|██████████| 503/503 [00:35<00:00, 14.33it/s]


## Screening Data

In [15]:
from stockMarket.stockScreener import StockScreener

screen_information = {
'dividendYield' : [0.1, None], # in percent
'price' : [None, 40],
'trailingPE' : [0, 30],
'forwardPE' : [0, 30],
'revenueGrowth' : [0, None], # in percent
'operatingCashflow' : [0, None], # in billions
'equityRatio' : [30, None], # in percent
'ebitdaMargin' : [15, None], # in percent
'minNetIncome' : [0, None], # in billions
}

stock_screener = StockScreener(stock_data)
selected_tickers = stock_screener.screen(screen_information)
# stock_screener.write_email(["benjaminlantschner@gmail.com", "97gamjak@gmail.com"])
# stock_screener.write_email(["97gamjak@gmail.com"])
# stock_screener.write_email(["benjaminlantschner@gmail.com"])


In [ ]:
from stockMarket.stockScreener import StockScreener

screen_information = {
'dividendYield' : [0.1, None], # in percent
'price' : [None, 40],
'trailingPE' : [0, 30],
'forwardPE' : [0, 30],
'revenueGrowth' : [0, None], # in percent
'operatingCashflow' : [0, None], # in billions
'equityRatio' : [30, None], # in percent
'ebitdaMargin' : [15, None], # in percent
'minNetIncome' : [0, None], # in billions
}

stock_screener = StockScreener(stock_data)
selected_tickers = stock_screener.continuous_screen(screen_information)
# stock_screener.write_email(["benjaminlantschner@gmail.com", "97gamjak@gmail.com"])
# stock_screener.write_email(["97gamjak@gmail.com"])
# stock_screener.write_email(["benjaminlantschner@gmail.com"])


In [14]:
print(stock_screener.summary)


A total of 24 out of 503 companies were selected.
For a total of 42 companies, data was missing.

A short summary of the reasons for discarding companies is given below:

dividendYield_lower_limit     0 with limits [0.1, None]
dividendYield_upper_limit     0 with limits [0.1, None]
price_lower_limit     0 with limits [None, 40]
price_upper_limit     1 with limits [None, 40]
trailingPE_lower_limit     0 with limits [0, 30]
trailingPE_upper_limit     0 with limits [0, 30]
forwardPE_lower_limit     0 with limits [0, 30]
forwardPE_upper_limit     0 with limits [0, 30]
revenueGrowth_lower_limit     0 with limits [0, None]
revenueGrowth_upper_limit     0 with limits [0, None]
operatingCashflow_lower_limit     0 with limits [0, None]
operatingCashflow_upper_limit     0 with limits [0, None]
equityRatio_lower_limit     0 with limits [30, None]
equityRatio_upper_limit     0 with limits [30, None]
ebitdaMargin_lower_limit     0 with limits [15, None]
ebitdaMargin_upper_limit     0 with limits [

In [20]:
from stockMarket.earning_dates import EarningDates

earning_dates = EarningDates(stock_data.tickers)
earning_dates.screen()
earning_dates.write_email(["benno94@virgilio.it", "97gamjak@gmail.com"])


Screening Long Name...



100%|██████████| 503/503 [00:00<00:00, 324076.02it/s]



Screening Sector...



100%|██████████| 503/503 [00:00<00:00, 629208.15it/s]



Screening Market Cap (B)...



100%|██████████| 503/503 [00:00<00:00, 463881.91it/s]



Screening Earnings Date...



100%|██████████| 503/503 [00:00<00:00, 446410.26it/s]